# Neural network classifier for rare disease prediction

Health claims data containing information on medical diagnoses, procedures and prescriptions can be used to identify undiagnosed rare disease patients. Ensemble decision tree approaches (such as `LightGBM` and `XGBoost`) are the most common machine learning approaches for tabular data. 

To optimise the performance of these models, feature engineering and feature selection is employed. Here, we implement a **Deep Neural Network classifier** optimised for the tabular datasets used in undiagnosed patient prediction in order to compare its performance to existing ensemble decision tree approaches.

In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import wandb
from wandb.keras import WandbCallback

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
%matplotlib inline 

In [ ]:
df = pd.read_csv('/home/cdsw/Data/EPI/imputed_dataset/test_epi_max_dd_imputed.csv')

### Preprocessing pipeline

In [ ]:
def preprocess(df, fraction=1, tt_split = 0.3):
  
  ## Discard missing values
  df = df.dropna()
  df = df.sample(frac=fraction)

  ## Define feature and target tables 
  targ_index = df.columns.get_loc('patient_type')
  X = df.iloc[:, 1:targ_index].values
  y = df.iloc[:, -1].values

  ## Class balancing with random undersampling
  from imblearn.under_sampling import RandomUnderSampler
  rus = RandomUnderSampler(random_state=0, ratio=0.1)
  X_resampled, y_resampled = rus.fit_resample(X, y)
  
  y_resampled= pd.get_dummies(y_resampled)

  ## One hot encode gender 
  onehotencoder = OneHotEncoder(categorical_features=[0])
  X_resampled = onehotencoder.fit_transform(X_resampled).toarray()

  X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=tt_split, 
                                                      shuffle= True)

  ##Feature scaling
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.fit_transform(X_test)
 
  return X_train, X_test, y_train, y_test 

In [ ]:
X_train, X_test, y_train, y_test = preprocess(df,1, 0.33)

## The neural network classifier

In [6]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.constraints import unit_norm
from keras.optimizers import SGD, Adadelta
from keras.callbacks import EarlyStopping
from sklearn.metrics import average_precision_score

#### Output activation function

`Softmax` function is used since outputs for this problem are mutually exclusive outputs.

#### Hyperparameters and tuning
The nn is built using the best hyperparameter values and architecture scanned with `RandomizedSearchCV`.

In [2]:
n_hidden=16
epochs=100
n_neurons=256
act_func='relu'
act_func_out='softmax'
opt = 'Adadelta'
kernel_init = 'glorot_uniform'
batch_size=1024
drop_rate=0.3

In [ ]:
model = Sequential()
model.add(Dense(n_neurons, input_shape=(X_train.shape[1],), activation=act_func,
                   kernel_initializer = kernel_init))
model.add(BatchNormalization())
model.add(Dropout(drop_rate))
# Add as many hidden layers as specified in nl
for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation=act_func, kernel_initializer = kernel_init))
    model.add(BatchNormalization())
    model.add(Dropout(drop_rate))
model.add(Dense(2, activation=act_func_out, kernel_initializer = kernel_init))

Define custom metrics for Keras.

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
model.compile(loss='categorical_crossentropy',optimizer=config.opt, metrics=['accuracy', f1_m,precision_m, recall_m])

In [ ]:
history = model.fit(X_train, y_train, epochs = config.epochs, validation_data =(X_test, y_test),
                    callbacks=[EarlyStopping(patience=50)])

### Evaluate
On training set:

In [ ]:
nn_pred_train = model.predict(X_test) #same as predict_proba

train_score = average_precision_score(y_test, nn_pred_train)
print('train set average precision score:',train_score)

On test set:

In [ ]:
df_test = pd.read_csv('/home/cdsw/Data/EPI/imputed_dataset/train_epi_max_dd_imputed.csv')
df_test = df_test.dropna()

targ_index = df_test.columns.get_loc('patient_type')
X_val = df_test.iloc[:, 1:targ_index]
y_val = df_test.iloc[:, -1].values

onehotencoder = OneHotEncoder(categorical_features=[0])
X_val_1hot = onehotencoder.fit_transform(X_val).toarray()
X_val = X_val_1hot

scaler = StandardScaler()
X_val = scaler.fit_transform(X_val)

y_val= pd.get_dummies(y_val)
nn_pred_val = model.predict(X_val)
test_score = average_precision_score(y_val, nn_pred_val)
print('test set average precision score:',test_score)